In [56]:
from os import (makedirs, path, listdir)
import random
from shutil import copy
import pandas as pd
DATA_ROOT = r'C:\Users\M AMIN\Downloads\birdsong-recognition'

In [61]:
def getdf():
    csv_path=path.join(DATA_ROOT,"train.csv")
    df=pd.read_csv(csv_path)
    df=df.drop(columns=[
        "rating","playback_used","channels",
        "pitch","speed", "description", "file_type", "volume", 
        "xc_id", "author", "url","length", "recordist",
        "title", "bird_seen", "sci_name", "location", "license"
    ])
    return df

In [81]:
def checkpaths():
    data_path=path.join(DATA_ROOT,"train_audio")
    df=getdf()
    df["path"]="temp"
    t=0
    for ind,rows in df.iterrows():
        t+=1
        rows['path']=path.join(data_path,rows["ebird_code"],rows["filename"])
        if not path.exists(rows["path"]):
            raise Exception("{} not found, please check path...".format(rows["path"]))
    return t

In [66]:
def totfiles():
    data_path=path.join(DATA_ROOT,"train_audio")
    t=0
    for birds in listdir(data_path):
        for samples in listdir(path.join(data_path,birds)):
            t+=1
    return t
totalfiles=totfiles()


In [84]:
from sklearn.model_selection import StratifiedKFold

In [103]:
skf = StratifiedKFold(5)
train_all=getdf()
train_all["fold"] = -1
for fold_id, (train_index, val_index) in enumerate(skf.split(train_all, train_all["ebird_code"])):
    train_all.iloc[val_index, -1] = fold_id
    
# # check the propotion
fold_proportion = pd.pivot_table(train_all, index="ebird_code", columns="fold", values="filename", aggfunc=len)
print(fold_proportion)

fold         0   1   2   3   4
ebird_code                    
aldfly      20  20  20  20  20
ameavo       8   8   8   7   7
amebit       9   9   8   9   9
amecro      20  20  20  20  20
amegfi      20  20  20  20  20
...         ..  ..  ..  ..  ..
yebsap      13  12  12  13  13
yehbla      11  12  12  11  11
yelwar      18  18  17  18  18
yerwar      20  20  20  20  20
yetvir      19  19  20  20  20

[264 rows x 5 columns]


In [99]:
len(train_all[train_all.fold==0])

4275

In [101]:
for fold_id, (train_index, val_index) in enumerate(skf.split(train_all, train_all["ebird_code"])):

0 <class 'numpy.ndarray'>
1 <class 'numpy.ndarray'>
2 <class 'numpy.ndarray'>
3 <class 'numpy.ndarray'>
4 <class 'numpy.ndarray'>


In [105]:
train_all.to_csv("train_with_folds")

Index(['ebird_code', 'date', 'duration', 'filename', 'species',
       'number_of_notes', 'secondary_labels', 'latitude', 'sampling_rate',
       'type', 'elevation', 'bitrate_of_mp3', 'background', 'country',
       'primary_label', 'longitude', 'time', 'resampled_sampling_rate',
       'resampled_filename', 'resampled_channels', 'fold'],
      dtype='object')

In [110]:
train_all.to_csv(r"C:\Users\M AMIN\Downloads\birdsong-recognition\train_with_folds.csv")

In [111]:
val=train_all[train_all["fold"]==4]
len(val)

4275

In [112]:
train=train_all[train_all["fold"]!=4]
len(train)

17100

In [124]:
train.iloc[95]

ebird_code                                                  ameavo
date                                                    2015-05-24
duration                                                        22
filename                                              XC304643.mp3
species                                            American Avocet
number_of_notes                                      Not specified
secondary_labels                                                []
latitude                                                   38.4716
sampling_rate                                           48000 (Hz)
type                                             call, flight call
elevation                                                    550 m
bitrate_of_mp3                                        128000 (bps)
background                                                     NaN
country                                              United States
primary_label              Recurvirostra americana_American Av

In [123]:
train_all.iloc[95]

ebird_code                                             aldfly
date                                               2018-06-12
duration                                                   25
filename                                         XC420402.mp3
species                                      Alder Flycatcher
number_of_notes                                 Not specified
secondary_labels                                           []
latitude                                              36.0973
sampling_rate                                      44100 (Hz)
type                                                     song
elevation                                              1900 m
bitrate_of_mp3                                   128000 (bps)
background                                                NaN
country                                         United States
primary_label              Empidonax alnorum_Alder Flycatcher
longitude                                            -82.1374
time    